In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import time
import numpy as np
import pandas as pd
import os.path
import scipy.spatial.distance as sd
from skip_thoughts import configuration
from skip_thoughts import encoder_manager
from scipy.spatial.distance import cosine, cityblock, jaccard, canberra, euclidean, minkowski, braycurtis

In [2]:
def load_model(uni = True):
    if uni:
        src = '/media/w/1c392724-ecf3-4615-8f3c-79368ec36380/DS Projects/Kaggle/Quora/scripts/models/v_tf-models/skip_thoughts/skip_thoughts/pretrained/skip_thoughts_uni_2017_02_02/'
        vocab = src + 'vocab.txt'
        embedding = src + 'embeddings.npy'
        check = src + 'model.ckpt-501424'
    else:
        src = '/media/w/1c392724-ecf3-4615-8f3c-79368ec36380/DS Projects/Kaggle/Quora/scripts/models/v_tf-models/skip_thoughts/skip_thoughts/pretrained/skip_thoughts_bi_2017_02_16/'
        vocab = src + 'vocab.txt'
        embedding = src + 'embeddings.npy'
        check = src + 'model.ckpt-500008'
    return vocab, embedding, check
    
def get_skipthoughts_vectors():
    st_src = '/media/w/1c392724-ecf3-4615-8f3c-79368ec36380/DS Projects/Kaggle/Quora/scripts/models/v_tf-models/skip_thoughts/'
    q1_enc = [x for x in os.listdir(st_src) if 'train_q1' in x]
    q2_enc = [x for x in os.listdir(st_src) if 'train_q2' in x]
    q1_vecs = np.zeros((0, 2400))
    for i in q1_enc:
        v = np.load(st_src + i)
        q1_vecs = np.concatenate((q1_vecs, v))
    q2_vecs = np.zeros((0, 2400))
    for i in q2_enc:
        v = np.load(st_src + i)
        q2_vecs = np.concatenate((q2_vecs, v))
    return q1_vecs.astype('float32'), q2_vecs.astype('float32')    


In [ ]:
vocab, embedding, check = load_model()

encoder = encoder_manager.EncoderManager()
encoder.load_model(configuration.model_config(),
                   vocabulary_file=vocab,
                   embedding_matrix_file=embedding,
                   checkpoint_path=check)

src = '/media/w/1c392724-ecf3-4615-8f3c-79368ec36380/DS Projects/Kaggle/Quora/scripts/features/'
trdf = pd.read_csv(src + 'df_train_spacylemmat_fullclean.csv')
trdf.fillna('NULL', inplace = True)

In [ ]:
for i in range(5):
    t = time.time()
    k = 100000
    ind_start = i*k
    ind_end = (i+1)*k
    print('Working on indexes: from {} to {}'.format(ind_start, ind_end))
    encodings1 = encoder.encode(trdf.question1[ind_start:ind_end])
    encodings2 = encoder.encode(trdf.question2[ind_start:ind_end])
    np.save('train_q1_skip_thoughts_encoding_part{}'.format(i), encodings1)
    np.save('train_q2_skip_thoughts_encoding_part{}'.format(i), encodings2)
    print('Time it took:', time.time() - t)

In [3]:
q1, q2 = get_skipthoughts_vectors()

In [4]:
t = time.time()

df = pd.DataFrame()
df['skipthoughts_cosine'] = [cosine(x, y) for (x, y) in zip(np.nan_to_num(q1), np.nan_to_num(q2))]
df['skipthoughts_jaccard'] = [jaccard(x, y) for (x, y) in zip(np.nan_to_num(q1), np.nan_to_num(q2))]
df['skipthoughts_canberra'] = [canberra(x, y) for (x, y) in zip(np.nan_to_num(q1), np.nan_to_num(q2))]
df['skipthoughts_euclidean'] = [euclidean(x, y) for (x, y) in zip(np.nan_to_num(q1), np.nan_to_num(q2))]
df['skipthoughts_minkowski'] = [minkowski(x, y, 3) for (x, y) in zip(np.nan_to_num(q1), np.nan_to_num(q2))]
df['skipthoughts_braycurtis'] = [braycurtis(x, y) for (x, y) in zip(np.nan_to_num(q1), np.nan_to_num(q2))]
df.to_csv('train_skipthoughts_distances.csv', index = False)

/home/w/anaconda3/envs/idp2/lib/python2.7/site-packages/scipy/spatial/distance.py:505: RuntimeWarning: invalid value encountered in true_divide
  dist = 1.0 - np.dot(u, v) / (norm(u) * norm(v))


In [5]:
df

,skipthoughts_cosine,skipthoughts_jaccard,skipthoughts_canberra,skipthoughts_euclidean,skipthoughts_minkowski,skipthoughts_braycurtis
0,0.644661,1.0,1525.923261,1.135483,0.385447,0.695880
1,0.890171,1.0,1701.289086,1.334295,0.447030,0.906312
2,0.714670,1.0,1600.597310,1.195550,0.401331,0.759321
3,0.710257,1.0,1609.797171,1.191854,0.401951,0.759701
4,0.793408,1.0,1614.792782,1.259689,0.420594,0.810954
5,0.918380,1.0,1710.844865,1.355271,0.450483,0.927785
6,0.744908,1.0,1599.320420,1.220580,0.417755,0.768596
7,0.909850,1.0,1724.741929,1.348962,0.451584,0.924239
8,0.817397,1.0,1631.555869,1.278590,0.429536,0.831138
9,0.869728,1.0,1664.566908,1.318884,0.444078,0.870838
